In [3]:
import pandas as pd
import numpy as np
import re
import pickle

ImportError: Unable to import required dependencies:
numpy: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [ ]:
df = pd.read_csv('song_lyrics_smaller.csv')

In [ ]:
df.drop(columns=['views','language_cld3', 'language_ft'], inplace=True)


In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
def clean_features(value):
    value = re.sub(r'[{}()"]', '', value)
    value = value.encode('utf-8').decode('utf-8', errors='ignore')
    if not value:
        return None
    return [v.strip().replace(" ", "") for v in value.split(',')]

In [ ]:
df['features'] = df['features'].apply(clean_features)

In [ ]:
df['N_artist'] = df['artist'].apply(clean_features)

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

In [ ]:
%pip install contractions
import contractions

In [ ]:
df.drop(df[df['language'] != 'en'].index, inplace=True)


In [ ]:
def modify(text):
    text = contractions.fix(text)
    text = text.replace('\\n', ' ')
    text = re.sub(r'\[.*?\]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))

    filtered_token = []
    for word in tokens:
        if word.lower() not in stop_words:
            filtered_token.append(word)

    token_no_pun = []
    for word in filtered_token:
        if word not in string.punctuation:
            token_no_pun.append(word)

    stemmer = PorterStemmer()
    stemmed_token = []
    for word in token_no_pun:
        stemmed_token.append(stemmer.stem(word))

    return stemmed_token

In [ ]:
df['modify_lyrics'] = df['lyrics'].apply(modify)

In [ ]:
df.head(10)

In [ ]:
with open('processed_data.pkl', 'wb') as file:
    pickle.dump(df, file)

In [ ]:
import pickle

In [ ]:
with open('processed_data.pkl', 'rb') as file:
    df = pickle.load(file)

In [ ]:
df[df['features'].apply(lambda x: x != [])]

In [ ]:
import ast

def convert_to_list(value):
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except:
            return []
    return value

df['features'] = df['features'].apply(convert_to_list)
df['N_artist'] = df['N_artist'].apply(convert_to_list)

In [ ]:
df['features'].apply(type)

In [ ]:
df['features'] = df['features'].apply(lambda x: x if isinstance(x, list) else [])


In [ ]:
df['combined'] = df.apply(lambda row: row['N_artist'] + row['features'], axis=1)


In [ ]:
df['combined']

In [ ]:
df['modify_lyrics'].apply(type).unique()

In [ ]:
df['combined_all'] = df['combined'].apply(lambda x: ' '.join(x)) + ' ' + df['tag'].apply(lambda x: ' '.join(x)) + ' ' + df['modify_lyrics'].apply(lambda x: ' '.join(x))


In [ ]:
df['combined_all'] = df['combined_all'].str.lower()

In [ ]:
df.head(1)

In [ ]:
df.drop(columns= ['N_artist','features','modify_lyrics','combined','year','id', 'language'], inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
vectorizer = TfidfVectorizer(max_features= 7000,stop_words='english')
vector = vectorizer.fit_transform(df['combined_all'])

In [ ]:
def recommend(song):
    if song not in df['title'].values:
        print(f"'{song}' is not found in the dataset.")
        return

    try:
        song_index = df[df['title'] == song].index[0]
        distances = cosine_similarity(vector[song_index].reshape(1, -1), vector)[0]
        songs_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[:21]
        print(f"Recommendations for '{song}':")

        for i in songs_list:
            if i[0] != song_index:
                print(df.iloc[i[0]].title)
    except Exception as e:
        print("An error occurred:", e)

In [ ]:
recommend('Raptured')

In [ ]:
import pickle

In [ ]:
pickle.dump(df.to_dict(),open('songs_dict.pkl', 'wb'))

In [ ]:
songs_dict = pickle.load(open('songs_dict.pkl', 'rb'))

In [ ]:
import pandas as pd
df = pd.DataFrame(songs_dict)

In [ ]:
df.head(10)